In [1]:
!pip install spacy_transformers
! pip install -U spacy

In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [8]:
cv_data = json.load(open('train_data_frv2.json'))

In [9]:
len(cv_data)

4449

In [10]:
!python -m spacy init fill-config base_config.cfg config.cfg

/Users/choukigueddar/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [12]:
len(train), len(test)

(3114, 1335)

In [13]:
type(train)

list

In [14]:
def get_spacy_doc(file, data):
    nlp = spacy.blank('fr')
    db = DocBin()

    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)

        # Vérifiez si annot est un dictionnaire et s'il contient la clé 'entities'
        if isinstance(annot, dict) and 'entities' in annot:
            annot_entities = annot['entities']
        else:
            # Gérez le cas où annot n'est pas conforme à vos attentes
            print(f"Warning: Invalid 'annot' format for text: {text}")
            continue

        ents = []
        entity_indices = []

        for start, end, label in annot_entities:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity:
                continue

            entity_indices += list(range(start, end))

            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except:
                continue

            if span is None:
                err_data = str([start, end]) + "     " + str(text) + "\n"
                file.write(err_data)
            else:
                ents.append(span)

        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass

    return db

In [15]:
file = open('error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

/Users/choukigueddar/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
100%|███████████████████████████████████| 3114/3114 [00:00<00:00, 281664.46it/s]


100%|███████████████████████████████████| 1335/1335 [00:00<00:00, 294379.68it/s]

In [16]:
len(db.tokens)

0

In [17]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 5.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [18]:
!python -m spacy train config.cfg --output ./output --paths.train train_data.spacy --paths.dev test_data.spacy

ℹ Saving to output directory: output
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
/Users/choukigueddar/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/choukigueddar/anaconda3/lib/python3.11/site-packages/spacy/__main__.py", line 4, in <module>
    setup_cli()
  File "/Users/choukigueddar/anaconda3/lib/python3.11/site-packages/spacy/cli/_util.py", line 87, in setup_cli
    command(prog_name=COMMAND)
  File "/Users/choukigueddar/anaconda3/lib/python3.11/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
 